## Projeto 6 - Classificação de imagens - Pytorch e Transfer Learning

        Let's Data - Jornada Cientista da Dados

**Motivação:** Estudar sobre utilização de redes pré-treinadas na classificação de imagens 

**Objeto de Estudo:** Na empresa fictícia Let's Veggie está com um problema na classificação dos produtos na loja. Muitos funcionarios não sabem diferenciar os vegetais e frutas, assim a empresa precisa de uma aplicação que faça essa classificação.

In [1]:
#!pip install torch torchvision pillow scikit-learn gradio

In [2]:
# Importando as bibliotecas 

from matplotlib import pyplot as plt
import numpy as np 
import os 
import PIL.Image #usamos para mostrar as imagens e ler elas 

import time 
import torch, torchvision 
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader # estrutura os dados para rede de treinamento 
import torch.nn as nn
import torch.optim as optim

In [3]:
# Separando as imagens em base de treino, validação e teste. Importante deixar uma pasta raw com os dados originais 

diretorio_base_imagens = "C:\\Users\\maria\\Projeto 6\\data\\raw"

# 'os.listdir' - Retorna uma lista contendo os nomes dos arquivos no diretório.
pasta_com_os_nomes_de_vegetais = os.listdir(diretorio_base_imagens)
pasta_com_os_nomes_de_vegetais

['batata', 'cenoura', 'limao', 'tomate']

In [4]:
# Separando a basa em 80% treino, 10% validação, 10% teste. Vamos criar uma separação estratificada 

quantidade_por_label = {}
for pasta in pasta_com_os_nomes_de_vegetais:
    # 'os.path.join' - une um ou mais componentes de caminho de forma inteligente.
    # '(os.listdir(os.path.join(diretorio_base_imagens,pasta)' - lista todas as imagens que tem dentro da pasta 
    quantidade_por_label[pasta] = len(os.listdir(os.path.join(diretorio_base_imagens,pasta)))

quantidade_por_label

{'batata': 146, 'cenoura': 181, 'limao': 111, 'tomate': 107}

In [5]:
#Criando uma pasta de treino, validação e teste

diretorio_imagens_processadas = "C:\\Users\\maria\\Projeto 6\\data\\processed"

dir_treino = os.path.join(diretorio_imagens_processadas, 'treino')
dir_validacao = os.path.join(diretorio_imagens_processadas, 'validacao')
dir_teste = os.path.join(diretorio_imagens_processadas, 'teste')

#verificando se as pastas foram criadas, senão foram 'os.makedirs' cria 

if not os.path.exists(dir_treino):
    os.makedirs(dir_treino)

if not os.path.exists(dir_validacao):
    os.makedirs(dir_validacao)
    
if not os.path.exists(dir_teste):
    os.makedirs(dir_teste)

In [6]:
# importando a biblioteca shutil para fazer copia de arquivo 

import shutil
from sklearn.model_selection import train_test_split

In [7]:
# Criando uma pasta para cada classe(batata, cenoura, limao, tomate) dentro de treino, validação e teste 

for classe in pasta_com_os_nomes_de_vegetais:
    
    dir_classe_treino = os.path.join(dir_treino, classe)
    dir_classe_validacao = os.path.join(dir_validacao, classe)
    dir_classe_teste = os.path.join(dir_teste, classe)
    
    if not os.path.exists(dir_classe_treino):
        os.makedirs(dir_classe_treino)
        
    if not os.path.exists(dir_classe_validacao):
        os.makedirs(dir_classe_validacao)
        
    if not os.path.exists(dir_classe_teste):
        os.makedirs(dir_classe_teste)
        
    # fazendo o caminho para a pasta com as imagens originais 
    pasta_classe = os.path.join(diretorio_base_imagens, classe)
    
    # listando todos os arquivos de imagem para essa clase 
    arquivos_classe = os.listdir(pasta_classe)
    
    # separando 80% treino e 20% validação/teste - Fazendo a separação em 80/20 pois 'train_test_split' separa apenas 2 
    # estamos passando uma lista com as imagens de cada classe, 'shuffle=True' ele vai embaralrar as imagens 
    treino, valid_test = train_test_split(arquivos_classe, shuffle=True, test_size=0.2, random_state=42)
    
    # Fazendo o 2 split no 'valid_test'. Agora o 'test_size' coloco 0.5 para divir em 50%
    validacao, teste = train_test_split(valid_test,shuffle=True,test_size=0.5,random_state=42)
    
    # Agora que tenho as variáveis de validação e teste não preciso mais da valid_test
    del valid_test
    
    print(f'{classe} - treino: {len(treino)} - validação: {len(validacao)} - teste: {len(teste)}')
    
    # Copiando os arquivos efetivamente para as pastas de treino, validação e teste 
    for imagem_treino in treino:
        caminho_origem = os.path.join(diretorio_base_imagens, classe, imagem_treino)
        caminho_destino = os.path.join(dir_classe_treino, imagem_treino)
        # estou pegando imagem por imagem caminho_origem (raw-'batata/tomante...'-imagem) e fazendo uma copia no 
        # caminho_destino (processed-treino-'batata/tomate...'-imagem)
        shutil.copy(caminho_origem, caminho_destino)
    
    
    for imagem_validacao in validacao:
        caminho_origem = os.path.join(diretorio_base_imagens, classe, imagem_validacao)
        caminho_destino = os.path.join(dir_classe_validacao, imagem_validacao)
       
        shutil.copy(caminho_origem, caminho_destino)
        
    
    for imagem_teste in teste:
        caminho_origem = os.path.join(diretorio_base_imagens, classe, imagem_teste)
        caminho_destino = os.path.join(dir_classe_teste, imagem_teste)
       
        shutil.copy(caminho_origem, caminho_destino)

batata - treino: 116 - validação: 15 - teste: 15
cenoura - treino: 144 - validação: 18 - teste: 19
limao - treino: 88 - validação: 11 - teste: 12
tomate - treino: 85 - validação: 11 - teste: 11


### Pré Processamento

In [8]:
# definido o tamanho da imagem para 100
imagem_size = 100 

# 'transforms' - transformando as imagens 
transformacao_de_imagem = {
    'treino': transforms.Compose([ # Compõe várias transformações juntas.
        transforms.Resize(size=[imagem_size, imagem_size]), # Redimensiona a imagem de entrada em tamanho especificado.100p/100
        transforms.ToTensor() # Converter um ``PIL Image`` ou ``numpy.ndarray`` para tensor.
     ]), 
    'validacao': transforms.Compose([
        transforms.Resize(size=[imagem_size,imagem_size]),
        transforms.ToTensor()
    ]),
    'teste': transforms.Compose([
        transforms.Resize(size=[imagem_size,imagem_size]),
        transforms.ToTensor()
    ])
}

Nessa transformação fizemos o redimensionamento das imagens e passamos para tensores Py Torch. Como não foi uma alteração significativa as mudanças vão ser feitas mas imagens de treinamento, validação e teste 

Caso tenha a necessidade de fazer mais alterações vamos fazer apenas no treino. Podemos fazer rotações, espelhamentos, crop randomicos... 

In [9]:
# Renomeando os nomes das pastas 

pasta_treino = dir_treino
pasta_validacao = dir_validacao
pasta_teste = dir_teste

**Preparação para o treinamento**

    Definido informações importantes para o modelo - Tamanho do batch, número de classes, datasets, dataloaders(organiza os dados para treinamento e validação para o treinamento da rede neural), otimizadores 

In [10]:
# Tamanho do batch (vamos treinar de 8 em 8 imagens)
tamanho_do_batch = 8 

#número de classes 
numero_classes = len(os.listdir(pasta_treino)) # vamos usar o len caso depois adicionarmos outra classe a variável atualiza 

In [11]:
# carregando as imagens usando o datasets do torchvision 

# 'ImageFolder' - Um carregador de dados genérico onde as imagens são organizadas dessa maneira por padrão
    # root= Caminho do diretório raiz
    # transform= Falo quais são as transfgormações que eu quero que sejam feitas 
data = {
    'treino': datasets.ImageFolder(root=pasta_treino, transform=transformacao_de_imagem['treino']),
    'validacao': datasets.ImageFolder(root=pasta_validacao, transform=transformacao_de_imagem['validacao'])
}

In [12]:
# associando os indices com os nomes das classes 

# estamos fazendo um dict comprehension, onde o indice é a chave e o nome da classe é o valor 
indice_para_classes = {indice: classes for classes, indice in data['treino'].class_to_idx.items()}

indice_para_classes

{0: 'batata', 1: 'cenoura', 2: 'limao', 3: 'tomate'}

In [13]:
# Quantidade de imagens para serem usadas para calcular o erro médio e a acurácia 

num_imagens_treino = len(data['treino'])
num_imagens_validacao = len(data['validacao'])

num_imagens_treino, num_imagens_validacao

(433, 55)

In [14]:
# Criando os DataLoader para o treino e validação 

data_loader_treino = DataLoader(data['treino'], batch_size=tamanho_do_batch,shuffle=True)
data_loader_validacao = DataLoader(data['validacao'], batch_size=tamanho_do_batch,shuffle=True)

### Transfer Learning 

Temos poucas imagens, dessa maneira fica dificil treinar um modelo de maneira eficiente. Assim vamos usar a Alexnet que tem milhares de imagem treinandas para ajudar nosso modelo. 

In [18]:
# Carregando Alexnet
# "weights=models.AlexNet_Weights.DEFAULT" falo que quero as imagens pré treinadas 

alexnet = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)

alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

Precisamos congelar os parametros da rede pré treinada, pois vou mudar a ultima camada para ele aprender as imagens que tenho salva, mas não quero que ele reaprenda todas as imgens que alexnet tem.  

In [19]:
# Precisamos congelar os parametros da rede pré-treinada 

for param in alexnet.parameters():
    # 'requires_grade' - desliga o treinamento e atualização dos pesos (coeficientes) das camadas da rede neural 
    param.requires_grade=False

In [21]:
# alterando a ultima camada. Era - (6): Linear(in_features=4096, out_features=1000, bias=True)
# e vai ficar (6): Linear(in_features=4096, out_features=4, bias=True) 
# Lembrando que tenho 4 classes(tomate, batata, limão, cenoura)
alexnet.classifier[6] = nn.Linear(4096, numero_classes)

# Adicionando um novo modulo - 'LogSoftmax' converte efetivamente em probabilidade para facilitar nossa análise 
alexnet.classifier.add_module("7", nn.LogSoftmax(dim=1))

alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [22]:
# Vamos utilizar a função de erro de entropia cruzada (comum para problemas de classificação)

funcao_erro = nn.CrossEntropyLoss()